In [1]:
# import libraries 
import numpy as np
import pandas as pd
import re
import math

In [2]:
# convert xlsx into dataframe, remove first row and headers
df_raw = pd.read_excel("NZ_Admin_JOBS.xlsx", skiprows = 1, header=None)

In [3]:
# column 3 replace empty cells with N/A
df_raw.fillna('N/A', inplace=True)

In [4]:
print(df_raw)

                                         0  \
0                            Administrator   
1                             Receptionist   
2             Prosecutions Support Officer   
3     Early Childhood Centre Administrator   
4           Business Support Administrator   
...                                    ...   
2703                   Key Account Manager   
2704                   Executive Assistant   
2705                Temporary Office Roles   
2706                Temporary Office Roles   
2707                   Executive Assistant   

                                                      1  \
0     https://www.seek.co.nz/job/50582301?type=promo...   
1     https://www.seek.co.nz/job/50620889?type=promo...   
2     https://www.seek.co.nz/job/50622169?type=stand...   
3     https://www.seek.co.nz/job/50639620?type=stand...   
4     https://www.seek.co.nz/job/50622432?type=stand...   
...                                                 ...   
2703  https://www.seek.co.nz/job/5

In [5]:
# column 4 remove duplications 
# get words and loop throughh all the words 
df_no_duplicates = df_raw

In [6]:
# Function to replace duplicated words in a string
def replace_duplicates(text):
    # split area from words 
    if 'area' in text :
        text = text.replace('area', ' area')
    # use regex to split the rest of the words in string
    words = re.sub(r'(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', ' ', text)
    # remove duplications
    unique_words = []
    words_list = words.split()
    for word in words_list :
        if word not in unique_words :
            unique_words.append(word)
    result = ' '.join(unique_words)
    return result

In [7]:
df_no_duplicates[3] = df_no_duplicates[3].apply(replace_duplicates)

In [8]:
print(df_no_duplicates)

                                         0  \
0                            Administrator   
1                             Receptionist   
2             Prosecutions Support Officer   
3     Early Childhood Centre Administrator   
4           Business Support Administrator   
...                                    ...   
2703                   Key Account Manager   
2704                   Executive Assistant   
2705                Temporary Office Roles   
2706                Temporary Office Roles   
2707                   Executive Assistant   

                                                      1  \
0     https://www.seek.co.nz/job/50582301?type=promo...   
1     https://www.seek.co.nz/job/50620889?type=promo...   
2     https://www.seek.co.nz/job/50622169?type=stand...   
3     https://www.seek.co.nz/job/50639620?type=stand...   
4     https://www.seek.co.nz/job/50622432?type=stand...   
...                                                 ...   
2703  https://www.seek.co.nz/job/5

In [9]:
# column 5: extract int value, standardize unit

In [10]:
# select cells that contains int values only
df_selected = df_no_duplicates[4][df_no_duplicates[4].str.contains(r'\d')]

In [11]:
# standardize units : replace hour and minutes to days 
df_selected = df_selected.str.replace(r'\d[hm]', '0d')

/var/folders/z2/206m_dxn0c327zkz616rnhph0000gp/T/ipykernel_7405/3167979754.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_selected = df_selected.str.replace(r'\d[hm]', '0d')


In [12]:
print(df_selected)

2                          4d ago,at
3                          0d ago,at
4       4d ago,at,Private Advertiser
5                          0d ago,at
6                          5d ago,at
                    ...             
2703                      27d ago,at
2704                      27d ago,at
2705                      20d ago,at
2706                      28d ago,at
2707                      26d ago,at
Name: 4, Length: 2688, dtype: object


In [13]:
# combine the modified column with original dataframe
df_standard_unit_column = df_selected.combine_first(df_no_duplicates[4])

In [14]:
print(df_standard_unit_column)

0       Featured,at,Private Advertiser
1                          Featured,at
2                            4d ago,at
3                            0d ago,at
4         4d ago,at,Private Advertiser
                     ...              
2703                        27d ago,at
2704                        27d ago,at
2705                        20d ago,at
2706                        28d ago,at
2707                        26d ago,at
Name: 4, Length: 2708, dtype: object


In [15]:
df_standard_unit = df_no_duplicates
df_standard_unit[4] = df_standard_unit_column

In [16]:
print(df_standard_unit)

                                         0  \
0                            Administrator   
1                             Receptionist   
2             Prosecutions Support Officer   
3     Early Childhood Centre Administrator   
4           Business Support Administrator   
...                                    ...   
2703                   Key Account Manager   
2704                   Executive Assistant   
2705                Temporary Office Roles   
2706                Temporary Office Roles   
2707                   Executive Assistant   

                                                      1  \
0     https://www.seek.co.nz/job/50582301?type=promo...   
1     https://www.seek.co.nz/job/50620889?type=promo...   
2     https://www.seek.co.nz/job/50622169?type=stand...   
3     https://www.seek.co.nz/job/50639620?type=stand...   
4     https://www.seek.co.nz/job/50622432?type=stand...   
...                                                 ...   
2703  https://www.seek.co.nz/job/5

In [17]:
# column 6: change to annual salary 

In [48]:
df_column5 = df_standard_unit[5]

In [49]:
print(df_column5)

0       classification: Administration & Office Suppor...
1       classification: Administration & Office Suppor...
2       classification: Administration & Office Suppor...
3       classification: Administration & Office Suppor...
4       classification: Administration & Office Suppor...
                              ...                        
2703    classification: Administration & Office Suppor...
2704                              Competitive hourly rate
2705                          Competitive hourly rates $$
2706                          Competitive hourly rates $$
2707    classification: Administration & Office Suppor...
Name: 5, Length: 2708, dtype: object


In [56]:
df_salary = df_column5[df_column5.str.contains(r'\$\d')]

In [57]:
print(df_salary)

12                                   $20 per hour
17                          $20 - $24.99 per hour
23                            $19 to $22 per hour
25                          $20 - $24.99 per hour
36                          $20 - $24.99 per hour
                          ...                    
679     $45,000 - $55,000 depending on experience
682              $24 - $28 p.h.  + 8% Holiday Pay
683             $35 - $40 p.h. + + 8% Holiday Pay
1936             $24 - $28 p.h.  + 8% Holiday Pay
1937            $35 - $40 p.h. + + 8% Holiday Pay
Name: 5, Length: 88, dtype: object


In [64]:
df_salary_hourly = df_salary[df_salary.str.contains(r'per hour|p\.h\.')]

In [89]:
print(df_salary_hourly)

12                                        $20 per hour
17                               $20 - $24.99 per hour
23                                 $19 to $22 per hour
25                               $20 - $24.99 per hour
36                               $20 - $24.99 per hour
48                                        $25 per hour
59                                     $20.00 per hour
66                               $20 - $24.99 per hour
77                                    $20.00 per hour.
78                               $20 - $24.99 per hour
81                                        $21 per hour
112                              $20 - $24.99 per hour
114                              $20 - $24.99 per hour
134                              $20 - $29.99 per hour
171                              $20 - $29.99 per hour
173                              $20 - $29.99 per hour
191                              $20 - $24.99 per hour
247                              $20 - $24.99 per hour
277       

In [114]:
def annual_salary_conversion(hourly):
    hourly = float(hourly)
    weekly_rate = hourly * 40  
    annualy_salary = weekly_rate * 48
    annualy_salary = round(annualy_salary, 2)
    return annualy_salary

In [115]:
def convert_hourly_to_annual(rate):
    rate = rate.replace('$', '').replace(',', '').replace('p.h.', '').replace('Up to', '').replace('per hour.', '').replace('per hour', '').replace('Holiday Pay','').replace('holiday pay','').replace('dependent on experience.','').replace('+','').replace('8%','')
    rate = rate.replace('to', '-').strip().split('-')
    
    if len(rate) == 1:
        hourly_rate = float(rate[0])
        annualy_rate = annual_salary_conversion(hourly_rate)
        result = f"${annualy_rate}"
        return result
    else:
        lower_limit = rate[0]
        upper_limit = rate[1]
        annual_lower_limit = annual_salary_conversion(lower_limit)
        annual_upper_limit = annual_salary_conversion(upper_limit)
        result = f"${annual_lower_limit} - ${annual_upper_limit}"
        return result

In [118]:
df_salary_annual = df_salary_hourly.apply(convert_hourly_to_annual)

In [119]:
print(df_salary_annual)

12                 $38400.0
17      $38400.0 - $47980.8
23      $36480.0 - $42240.0
25      $38400.0 - $47980.8
36      $38400.0 - $47980.8
48                 $48000.0
59                 $38400.0
66      $38400.0 - $47980.8
77                 $38400.0
78      $38400.0 - $47980.8
81                 $40320.0
112     $38400.0 - $47980.8
114     $38400.0 - $47980.8
134     $38400.0 - $57580.8
171     $38400.0 - $57580.8
173     $38400.0 - $57580.8
191     $38400.0 - $47980.8
247     $38400.0 - $47980.8
277                $42240.0
282     $38400.0 - $47980.8
349     $38400.0 - $48000.0
379     $38400.0 - $47980.8
384     $57600.0 - $72960.0
403                $44160.0
405                $42240.0
458     $51840.0 - $63360.0
464     $36480.0 - $42240.0
478     $67200.0 - $76780.8
494     $38400.0 - $47980.8
503                $51840.0
517     $38400.0 - $47980.8
536     $40320.0 - $48000.0
550     $40320.0 - $44160.0
560     $76800.0 - $86400.0
569     $35520.0 - $45120.0
578                $

In [120]:
df_salary_column = df_salary_annual.combine_first(df_column5)

In [123]:
df_salary = df_standard_unit

In [125]:
df_salary[5] = df_salary_column

In [126]:
print(df_salary)

                                         0  \
0                            Administrator   
1                             Receptionist   
2             Prosecutions Support Officer   
3     Early Childhood Centre Administrator   
4           Business Support Administrator   
...                                    ...   
2703                   Key Account Manager   
2704                   Executive Assistant   
2705                Temporary Office Roles   
2706                Temporary Office Roles   
2707                   Executive Assistant   

                                                      1  \
0     https://www.seek.co.nz/job/50582301?type=promo...   
1     https://www.seek.co.nz/job/50620889?type=promo...   
2     https://www.seek.co.nz/job/50622169?type=stand...   
3     https://www.seek.co.nz/job/50639620?type=stand...   
4     https://www.seek.co.nz/job/50622432?type=stand...   
...                                                 ...   
2703  https://www.seek.co.nz/job/5

$38400.0
